# mipt_dl_captcha_2025

Данные для обучения содержатся в двух файлах: `images.npy` и `labels.npy`. Это формат numpy-массивов.

```#загрузка данных
import numpy as np
images = np.load('/kaggle/input/mipt-dl-captcha/data/images.npy')
labels = np.load('/kaggle/input/mipt-dl-captcha/data/labels.npy')
```  
Всего доступно 20 тысяч размеченных изображений размером 48x48 с 3 цветовыми каналами (RGB), поэтому массив _images_ имеет размер (20000, 48, 48, 3). В массиве _labels_ содержатся ответы к тренировочному набору изображений. В английском алфавите 26 букв: 0-й класс соответствует букве А и так далее по алфавиту, 25-й класс — буква Z.  


Ваша задача — обучить нейронную сеть и с ее помощью предсказать метки классов для изображений из файла _images_sub.npy_, в нем 50 тысяч изображений. Посмотрите на структуру файла _sample_submission.csv_ — он не содержит полезных данных, а лишь описывает формат, в котором вы загружаете ваши предсказания на сайт. Создайте из ваших предсказаний такой же файл и загрузите в качестве вашего ответа (сабмита). Вы можете делать до 20 сабмитов в сутки.
Метрика лидерборда — __accuracy__, то есть доля правильно распознанных изображений.
  
Задача будет считаться решенной, если в Public Leaderboard вы наберете score, который будет равен или больше __0.82__.

# Dataset Description

`images.npy` - изображения для обучения
`labels.npy` - метки для обучения
`sample_submission.csv` - формат файла для сабмита
`images_sub.npy` - изображения для сабмита
Файлы `.npy` - это сохраненные numpy-массивы. Их можно загрузить с помощью функции `np.load(filename)`

In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
# Загрузка данных
images = np.load('../mds-misis-dl-captchan/images.npy')
labels = np.load('../mds-misis-dl-captchan/labels.npy')

print(f'{images.shape = }') if images.size != 0 else print('images не загружен')
print(f'{labels.shape = }') if labels.size != 0 else print('labels не загружен')

images.shape = (20000, 48, 48, 3)
labels.shape = (20000,)


In [3]:
# Сразу нормализуем данные
images_norm = images / 255


In [4]:
try:
    # Преобразование данных в тензор
    X = torch.tensor(images_norm,dtype=torch.float32).permute(0, 3, 1, 2)
    y = torch.tensor(labels, dtype=torch.long)

    # Стандартизируем данные:
    # Значения для нормализации
    mean = torch.tensor([0.5, 0.5, 0.5])
    std = torch.tensor([0.5, 0.5, 0.5])

    X = (X - mean[None, :, None, None]) / std[None, :, None, None]

    # Разбиваем данные на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Создаём датасеты для работы с pytorch
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

    # Разбиваем датасеты на батчи
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print('Объекты для обучения созданы успешно!')
except Exception as e:
    print(f'Не удалось сформировать объекты для обучения: {e}')

Объекты для обучения созданы успешно!


In [5]:
# Переносим вычисления на gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

Using cuda device


In [6]:
# Базовая нейронная сеть
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(48*48*3, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 26)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=6912, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=2048, bias=True)
    (5): ReLU()
    (6): Linear(in_features=2048, out_features=26, bias=True)
  )
)


In [7]:
# Пишем свёрточную нейроннуя сеть
class ConvNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv_stack = nn.Sequential(
            # Из 3-х каналов делаем 32 фильтра
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            # Уменьшаем размерность данных до 24*24
            nn.MaxPool2d(2, 2),
            # 32 -> 64 фильтра
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Уменьшаем размерность до 12*12
            nn.MaxPool2d(2, 2),
            # 64 -> 128 фильтров
            nn.Conv2d(64, 128 ,kernel_size=3, padding=1),
            nn.ReLU(),
            # Размерность 12*12 -> 6*6
            nn.MaxPool2d(2, 2)
        )
        self.linear_stack = nn.Sequential(
            # Входной массив 128 фильтров для матрицы 6*6
            nn.Linear(128*6*6, 512),
            nn.ReLU(),
            # Регуляризация
            nn.Dropout(0.3),
            # Целевой размер - 26 по количеству таргетных лейблов (букв)
            nn.Linear(512, 26)
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits


model = ConvNeuralNetwork().to(device)
print(model)

ConvNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv_stack): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_stack): Sequential(
    (0): Linear(in_features=4608, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=26, bias=True)
  )
)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    # Итерируемся по батчам даталоадера
    for batch, (X, y) in enumerate(dataloader):
        # Переносим данные на gpu
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Вычисляем градиенты по отношению к параметрам модели
        loss.backward()
        # Обнуляем градиенты для следующего шага
        optimizer.step()
        optimizer.zero_grad()
        
        # Логируем процесс обучения каждые 100 батчей
        # if batch % 100 == 0:
        #     loss, curr = loss.item(), (batch + 1) * len(X)
        #     print(f'loss = {loss:>7f} [{curr:>5d}/{size:>5d}]')

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    # Отключаем вычисление градиентов
    with torch.no_grad():
        # Итерируемся по батчам тестовых данных
        for X, y in dataloader:
            # Переносим данные на gpu
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()

    # Средний loss на тестовых данных
    test_loss /= num_batches
    # Общее количество верных предсказаний
    accuracy = correct / size

    print(f'Test error:\n\tAccuracy: {(accuracy * 100):>0.1f}%,\n\tAvg loss: {test_loss:>8f}')

    # Возвращаем accuracy для дальнейшего сохранения лучшей модели
    return accuracy

In [11]:
# Параметры для информативного сохранения модели
checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'best_accuracy': best_accuracy
}

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

# Обучим модель на 50 эпохах + сохраним лучшую модель
num_epochs = 50
best_accuracy = 0.0
patience = 10
counter = 0
best_model_path = 'best_model.pth'

for epoch in range(num_epochs):
    # Обучение
    print(f'Epoch {epoch + 1}\n', '='*20)
    train(train_data_loader, model, loss_fn, optimizer)
    accuracy = test(test_data_loader, model, loss_fn)

    # Сохраним лучшую модель
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        counter = 0
        torch.save(checkpoint, best_model_path)
        print(f'Сохранена модель с лучшим accuracy: {best_accuracy * 100:.1f}%')
    else:
        counter += 1

    # Ранняя остановка, если модель долго не улучшает точность
    if counter > patience:
        print(f'Ранняя остановка на {epoch + 1} эпохе')
        break

print('Done!')

Epoch 1
Test error:
	Accuracy: 83.1%,
	Avg loss: 0.882065
Сохранена модель с лучшим accuracy: 83.1%
Epoch 2
Test error:
	Accuracy: 81.5%,
	Avg loss: 0.933313
Epoch 3
Test error:
	Accuracy: 83.0%,
	Avg loss: 0.903506
Epoch 4
Test error:
	Accuracy: 82.4%,
	Avg loss: 0.891160
Epoch 5
Test error:
	Accuracy: 82.2%,
	Avg loss: 0.953009
Epoch 6
Test error:
	Accuracy: 82.6%,
	Avg loss: 0.914104
Epoch 7
Test error:
	Accuracy: 82.2%,
	Avg loss: 0.941656
Epoch 8
Test error:
	Accuracy: 82.8%,
	Avg loss: 0.910129
Epoch 9
Test error:
	Accuracy: 83.1%,
	Avg loss: 0.912534
Сохранена модель с лучшим accuracy: 83.1%
Epoch 10
Test error:
	Accuracy: 83.9%,
	Avg loss: 0.861640
Сохранена модель с лучшим accuracy: 83.9%
Epoch 11
Test error:
	Accuracy: 82.8%,
	Avg loss: 0.898419
Epoch 12
Test error:
	Accuracy: 83.3%,
	Avg loss: 0.893004
Epoch 13
Test error:
	Accuracy: 82.6%,
	Avg loss: 0.951562
Epoch 14
Test error:
	Accuracy: 83.3%,
	Avg loss: 0.899114
Epoch 15
Test error:
	Accuracy: 83.0%,
	Avg loss: 0.93011

# Предсказания

In [12]:
# Загрузка данных для предсказания
data = np.load('../mds-misis-dl-captchan/images_sub.npy')

data.shape

(50000, 48, 48, 3)

In [14]:
# Нормализуем и стандартизируем данные
data_norm = data / 255

In [15]:
# Преобразуем в тензор и стандартизируем
test_X = torch.tensor(data_norm, dtype=torch.float32).permute(0, 3, 1, 2)

mean = torch.tensor([0.5, 0.5, 0.5])
std = torch.tensor([0.5, 0.5, 0.5])

test_X = (test_X - mean[None, :, None, None]) / std[None, :, None, None]

# Преобразуем в torch датасет и даталоадер
subm_dataset = torch.utils.data.TensorDataset(test_X)
subm_data_loader = torch.utils.data.DataLoader(subm_dataset, batch_size=32, shuffle=False)

In [20]:
# Загрузим модель
try:
    best_model = torch.load('best_model.pth', map_location=device)
    model.load_state_dict(best_model['state_dict'])
    model.eval()
    print(f'Успешно загружена модель с заявленной accuracy: {best_model['best_accuracy']}')
except Exception as e:
    print('Не удалось загрузить модель')
    print(f'Ошибка: {e}')

Успешно загружена модель с заявленной accuracy: 0.8315


In [21]:
# Получение предсказаний
predictions = []
with torch.no_grad():
    for batch in subm_data_loader:
        X = batch[0].to(device)
        outputs = model(X)
        preds = outputs.argmax(dim=1).cpu().numpy()
        predictions.extend(preds)

In [22]:
# Формирование массива Id,Category
ids = np.arange(len(predictions))
results = np.column_stack((ids, predictions))
print(results.shape)

(50000, 2)


In [24]:
# Сохраним результаты в .csv
results_df = pd.DataFrame(results, columns=['Id', 'Category'])
results_df.to_csv('submission.csv', index=False)
print('Предсказания сохранены в файл submission.csv')

Предсказания сохранены в файл submission.csv


In [25]:
display(results_df)

,Id,Category
0,0,19
1,1,2
2,2,14
3,3,12
4,4,10
...,...,...
49995,49995,5
49996,49996,10
49997,49997,14
49998,49998,3
